In [ ]:
# Решение задания №1

!pip install pymorphy2 nltk
import nltk
nltk.download("punkt")
nltk.download("stopwords")

!pip install gensim

from google.colab import drive
import os
drive.mount("/content/drive/", force_remount=True)

In [ ]:
import pandas as pd

columns = ['id', 'date', 'name', 'text', 'positive', 'rep', 'rtv', 'fav', 'total_count', 'fol', 'friends', 'lisy_count']
df_positive = pd.read_csv('/content/drive/MyDrive/school21/positive.csv', sep = ";", names = columns)
df_negative = pd.read_csv('/content/drive/MyDrive/school21/negative.csv', sep = ";", names = columns)
df = pd.concat((df_positive, df_negative), axis=0)
df.positive[df.positive == -1] = 0
df.index = list(range(226834))

In [ ]:
df.text = df.text.str.lower()
df.text = df.text.str.replace(r"[^А-Яа-я]"," ")
df.text.loc[19:22]

In [ ]:
# Сделаем токенизацию текста и удалим стоп-слова:

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

df.text = list(map(word_tokenize, df.text))
russian_stopwords = stopwords.words("russian")
russian_stopwords.sort()
russian_stopwords

def delete_stopword(words):
    global russian_stopwords
    new_s = [word for word in words if word not in russian_stopwords]
    return new_s

df.text = list(map(delete_stopword, df.text))
df.text.loc[19:22]

In [ ]:
# Проведем лемматизацию полученных слов:

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatization(words):
    global morph
    new_s = [morph.parse(word)[0].normal_form for word in words]
    return new_s

df.text = list(map(lemmatization, df.text))
df.text.loc[19:22]

In [ ]:
preprocessed_df = df[["text", "positive"]]
preprocessed_df

In [ ]:
from gensim.models import Word2Vec

w2v_10_10 = Word2Vec(size=10, min_count=10)
w2v_10_10.build_vocab(preprocessed_df.text)  # создадим словарь
w2v_10_10.train(preprocessed_df.text, total_examples=w2v_10_10.corpus_count, epochs=100) # обучим модель на нашем наборе текстов
w2v_10_10.save('w2v_10_10')

# w2v_300_10 = Word2Vec(size=300, min_count=10) 
# w2v_300_10.build_vocab(preprocessed_df.text)
# w2v_300_10.train(preprocessed_df.text, total_examples=w2v_300_10.corpus_count, epochs=100)
# w2v_300_10.save('w2v_300_10')

w2v_500_10 = Word2Vec(size=500, min_count=10) 
w2v_500_10.build_vocab(preprocessed_df.text)
w2v_500_10.train(preprocessed_df.text, total_examples=w2v_500_10.corpus_count, epochs=100)
w2v_500_10.save('w2v_500_10')

w2v_300_1 = Word2Vec(size=300, min_count=1) 
w2v_300_1.build_vocab(preprocessed_df.text)
w2v_300_1.train(preprocessed_df.text, total_examples=w2v_300_1.corpus_count, epochs=100)
w2v_300_1.save('w2v_300_1')

w2v_300_100 = Word2Vec(size=300, min_count=100) 
w2v_300_100.build_vocab(preprocessed_df.text)
w2v_300_100.train(preprocessed_df.text, total_examples=w2v_300_100.corpus_count, epochs=100)
w2v_300_100.save('w2v_300_100')

w2v_10_10_n = Word2Vec.load('w2v_10_10')
# w2v_300_10_n = Word2Vec.load('w2v_300_10')
w2v_500_10_n = Word2Vec.load('w2v_500_10')
w2v_300_1_n = Word2Vec.load('w2v_300_1')
w2v_300_100_n = Word2Vec.load('w2v_300_100')


In [ ]:
w2v_300_10 = Word2Vec(size=300, min_count=10) 
w2v_300_10.build_vocab(preprocessed_df.text)
w2v_300_10.train(preprocessed_df.text, total_examples=w2v_300_10.corpus_count, epochs=100)
w2v_300_10.save('w2v_300_10')

w2v_300_10_n = Word2Vec.load('w2v_300_10')

In [ ]:
# Выберем случайные слова из датафрейма
import random

set = []

# Выберем пять позитивных слов
for i in range(0, 5):
  str_p = random.choice(list(preprocessed_df[df.positive == 1].text))
  word = random.sample(str_p, 1)
  set.append(word)

# Выберем пять негативных слов
for i in range(0, 5):
  str_p = random.choice(list(preprocessed_df[df.positive == 0].text))
  word = random.sample(str_p, 1)
  set.append(word)

set

In [ ]:
# Синонимы для модели w2v_300_1:

for i in range(0, 10):
  w2v_300_1.wv.most_similar(positive=set[i], topn=15)
  #print(w2v_300_1.wv.most_similar(positive=set[i], topn=15))

# Антонимы для модели w2v_300_1:

for i in range(0, 10):
  w2v_300_1.wv.most_similar(negative=set[i], topn=15)
  #print(w2v_300_1.wv.most_similar(negative=set[i], topn=15))

In [ ]:
# Синонимы для модели w2v_300_10:

for i in range(0, 10):
  w2v_300_10.wv.most_similar(positive=set[i], topn=15)
  #print(w2v_300_10.wv.most_similar(positive=set10[i], topn=15))

# Антонимы для модели w2v_300_10:

for i in range(0, 10):
  w2v_300_10.wv.most_similar(negative=set[i], topn=15)
  #print(w2v_300_10.wv.most_similar(negative=set10[i], topn=15))

In [ ]:
set100 = [['сейчас'],
 ['папа'],
 ['забавный'],
 ['против'],
 ['нужный'],
 ['мой'],
 ['никто'],
 ['наушник'],
 ['есть'],
 ['быть']]

In [ ]:
# Синонимы для модели w2v_300_100:

for i in range(0, 10):
  w2v_300_100.wv.most_similar(positive=set100[i], topn=15)
  #print(w2v_300_100.wv.most_similar(positive=set100[i], topn=15))

# Антонимы для модели w2v_300_100:

for i in range(0, 10):
  w2v_300_100.wv.most_similar(negative=set100[i], topn=15)
  #print(w2v_300_100.wv.most_similar(negative=set100[i], topn=15))

In [ ]:
# Синонимы для модели w2v_10_10:

for i in range(0, 10):
  w2v_10_10.wv.most_similar(positive=set[i], topn=15)
  #print(w2v_10_10.wv.most_similar(positive=set[i], topn=15))

# Антонимы для модели w2v_10_10:

for i in range(0, 10):
  w2v_10_10.wv.most_similar(negative=set[i], topn=15)
  #print(w2v_10_10.wv.most_similar(negative=set[i], topn=15))

In [ ]:
# Синонимы для модели w2v_500_10:

for i in range(0, 10):
  w2v_500_10.wv.most_similar(positive=set[i], topn=15)
  #print(w2v_500_10.wv.most_similar(positive=set10[i], topn=15))

# Антонимы для модели w2v_500_10:

for i in range(0, 10):
  w2v_500_10.wv.most_similar(negative=set[i], topn=15)
  #print(w2v_500_10.wv.most_similar(negative=set10[i], topn=15))

Вывод.

Чем больше размер результирующего пространства, тем выше точность определения синонимов/антонимов моделью потому, что у модели больше критериев оценки близости слов и можно это сделать точнее.

Минимальная встречаемость слов влияет на точность определения синонимов/антонимов моделью так: чем больше минимальная встречаемость слов, тем больше точность определения синонимов/антонимов. Тем не менее, в результат могут не попасть слова, являющиеся наиболее точными синонимами/антонимамим, но не попавшие в выборку должное количество раз.

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = (15, 10)


def reduce_dimensions(w2v_model):
    """Фукнция принимает модель word2vec и возвращает массив абсцисс,
    массив ординат и массив слов после снижения размерности"""
    tsne = TSNE(n_components=2, random_state=256)  # создадим экземпляр модели TSNE
    vectors = np.asarray(w2v_model.wv.vectors)     # возьмем из модели 300-мерный массив слов-векторов
    labels = np.asarray(w2v_model.wv.index2word)   # отдельно сохраним соответствие номера вектора и самого слова
    vectors = tsne.fit_transform(vectors)          # проведем преобразование каждого вектора в 2-мерный

    x = [v[0] for v in vectors]                    # запишем отдельно массив абсцисс и массив ординат
    y = [v[1] for v in vectors]
    return x, y, labels


def plot_w2v(w2v_model):
    """Функция строит график распределения слов по векторному пространству
    размерности 2 исходя из обученной модели word2vec"""
    x, y, labels = reduce_dimensions(w2v_model)                      # получим значения по осям и названия точек (исходные слова)
    plt.scatter(x, y)                                                # строим график с точками
    words_to_show_indices = np.random.randint(len(labels), size=25)  # выберем 25 случайных слов, которые отобразим на графике
    for i in words_to_show_indices:
        plt.annotate(labels[i], (x[i], y[i]))                        # для каждого из этих 25 слов отобразим текст на картинке
    plt.show()

plot_w2v(w2v_10_10)

In [ ]:
plot_w2v(w2v_300_10)

In [ ]:
plot_w2v(w2v_500_10)

In [ ]:
plot_w2v(w2v_300_1)

In [ ]:
plot_w2v(w2v_300_100)

Вывод.

Чем больше минимальная встречаемость слов, тем реже расположены точки на графике. Это происходит в первую очередь потому, что в рассчет берется меньшее количество слов.

Чем больше размер результирующего пространства, тем более явно наблюдается уплотнение точек ближе к центру. Это связано с тем, что с увеличением размера результирующего пространства повышается точность определения связей между словами. Большой размер результирующего пространства может вызывать переобученность.

In [ ]:
from gensim.models import Word2Vec

w2v = Word2Vec(size=300, min_count=2) 
w2v.build_vocab(preprocessed_df.text)

Обучим модель по умолчанию и попробуем предсказать продолжение случайной фразы:

In [ ]:
w2v.train(preprocessed_df.text, total_examples=preprocessed_df.shape[0], epochs=100)
w2v.predict_output_word(["такси", "везти", "работа"])

In [ ]:
w2v.train(preprocessed_df.text, total_examples=preprocessed_df.shape[0], epochs=10)
w2v.predict_output_word(["такси", "везти", "работа"])

Попробуем предсказать продолжение твита "Котёнка вчера носик разбила, плакала и расстраивалась :("

In [ ]:
w2v.train(preprocessed_df.text, total_examples=preprocessed_df.shape[0], epochs=10)
w2v.predict_output_word(["вчера", "носик", "разбить"])

In [ ]:
w2v.train(preprocessed_df.text, total_examples=preprocessed_df.shape[0], epochs=100)
w2v.predict_output_word(["вчера", "носик", "разбить"])

Чем больше эпох обучения, тем более точное предсказание дает модель.

In [ ]:
# Решение задания №2

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
!pip install xgboost

tweets = [TaggedDocument(doc, [i]) for i, doc in enumerate(preprocessed_df.text)]

# Разобьем набор текстов на тренировочную и тестовую выборки
X_train_texts, X_test_texts, y_train, y_test = train_test_split(preprocessed_df.text, preprocessed_df.positive, test_size=0.2, random_state=21)

In [ ]:
# Создадим модель Doc2Vec с минимальной встречаемостью слова 2
d2v = Doc2Vec(tweets, min_count=2)
d2v.train(tweets, total_examples=len(tweets), epochs=20)  # подберем веса коэффициентов внутри модели, которые больше будут подходить к нашему набору текстов

In [ ]:
def transform_text_array_to_vector_dataframe(text_array):
    """Функция, которая преобразует одномерную колонку списков слов из текстов
    в датафрейм со значениями векторов этих текстов"""
    columns = [str(n) for n in range(d2v.vector_size)]               # задаем список названий колонок - просто порядковые номера
    vectors_ndarray = text_array.apply(d2v.infer_vector).to_list()  # прогоняем каждый текст через модель doc2vec и формируем многомерный массив чисел
    return pd.DataFrame(vectors_ndarray, columns=columns)            # оборачиваем его в датафрейм для удобства


X_train = transform_text_array_to_vector_dataframe(X_train_texts)    # наконец создадим датафреймы, которые сможем подать в модель классификации
X_test = transform_text_array_to_vector_dataframe(X_test_texts)

In [ ]:
# Применим модель xgboost. Основные гиперпараметры: max_depth - максимальная глубина деревьев модели и n_estimators - количество деревьев.
# min_count=2, epochs=20

from xgboost import XGBClassifier
from sklearn.metrics import classification_report

xgb = XGBClassifier(max_depth=10, n_estimators=50)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

Попробуем поменять параметр минимальная встречаемость слова в текстах в doc2vec

In [ ]:
# Попробуем минимальную встречаемость 10
# min_count=10, epochs=20

d2v = Doc2Vec(tweets, min_count=10)
d2v.train(tweets, total_examples=len(tweets), epochs=20)

X_train10 = transform_text_array_to_vector_dataframe(X_train_texts)
X_test10 = transform_text_array_to_vector_dataframe(X_test_texts)

xgb.fit(X_train10, y_train)
y_pred = xgb.predict(X_test10)
print(classification_report(y_pred, y_test))

In [ ]:
# Попробуем минимальную встречаемость 100
# min_count=100, epochs=20

d2v = Doc2Vec(tweets, min_count=100)
d2v.train(tweets, total_examples=len(tweets), epochs=20)

X_train100 = transform_text_array_to_vector_dataframe(X_train_texts)
X_test100 = transform_text_array_to_vector_dataframe(X_test_texts)

xgb.fit(X_train100, y_train)
y_pred = xgb.predict(X_test100)
print(classification_report(y_pred, y_test))

Попробуем разные модели, меняя гиперпараметры максимальная глубина деревеьев в бустинге и количество деревьев в бустинге

In [ ]:
# Применим параметры, общие для всех min_count=10, epochs=10

d2v = Doc2Vec(tweets, min_count=10)
d2v.train(tweets, total_examples=len(tweets), epochs=10)

X_train = transform_text_array_to_vector_dataframe(X_train_texts)
X_test = transform_text_array_to_vector_dataframe(X_test_texts)

In [ ]:
xgb = XGBClassifier(max_depth=5, n_estimators=50)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

In [ ]:
xgb = XGBClassifier(max_depth=10, n_estimators=50)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

In [ ]:
xgb = XGBClassifier(max_depth=10, n_estimators=10)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

In [ ]:
xgb = XGBClassifier(max_depth=10, n_estimators=100)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

Вывод.

Лучше использовать более глубокие деревья в модели градиентного бустинга. Увеличение количества деревьев также улучшает результат